In [1]:
import pandas as pd
import json
import numpy as np
from glob import glob
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib

matplotlib.rc('pdf', fonttype=42)
matplotlib.rc('ps', fonttype=42)

import os
sns.set(font_scale=1.25, style='whitegrid')
sns.set(palette='magma')

# rdir = '../results/'
rdir = '../results_blackbox/'
# figdir = 'figs/black-box' 
figdir = '../paper/'
# Set to path to PMLB datasets
datadir = '../datasets/pmlb/datasets/'
print('figdir:',figdir)
def save(name='tmp',h=None):
    name = name.strip().replace(' ','-').replace('%','pct')
    if h == None:
        h = plt.gcf()
    h.tight_layout()
    print('saving',figdir+'/'+name+'.pdf')
    if not os.path.exists(figdir):
        os.makedirs(figdir)
    plt.savefig(figdir+'/'+name+'.pdf', bbox_inches='tight')
    

figdir: ../paper/


In [2]:
##########
# load results
##########
df_results = pd.read_feather('../results/black-box_results_srbench.feather')

df_results = pd.concat([pd.read_feather('../results/black-box_results_brush_250.feather'), df_results], axis=0)
# df_results = pd.concat([pd.read_feather('../results/black-box_results_pstree.feather'), df_results], axis=0)

# df_results.reset_index()

# Removing mislabeled datasets (these are clf, but PMLB v1.0 had it as regr)
df_results = df_results[ ~df_results["dataset"].isin(["banana", "titanic"]) ]


In [3]:
df_results.columns

Index(['dataset', 'algorithm', 'random_state', 'training time (s)',
       'symbolic_model', 'mse_test', 'mae_test', 'r2_test', 'model_size',
       'params_str', 'training time (hr)', 'r2_zero_test', 'friedman_dataset',
       'symbolic_alg'],
      dtype='object')

In [7]:
for brush in ['Brush',
    'Brush wo split',
    'Brush wo split (D-UCB1)',
    "Brush (D-UCB1)",
]:
    df_aux = df_results[
        (df_results['algorithm']==brush)
        & (df_results['symbolic_model'].str.contains("If"))
    ]
    perc = df_aux.shape[0]/df_results[df_results['algorithm']==brush].shape[0]
    print(brush, df_aux.shape[0], perc)


Brush 158 0.1302555647155812
Brush wo split 0 0.0
Brush wo split (D-UCB1) 0 0.0
Brush (D-UCB1) 167 0.1374485596707819


In [5]:
with pd.option_context('display.max_colwidth', None):
    df_aux = df_results[
        (df_results['algorithm']=='Brush (D-UCB1)')
        & (df_results['symbolic_model'].str.contains("If"))
    ][['dataset', 'r2_test', 'mse_test', 'symbolic_model']]

    df_aux.to_csv("./inspecting_models.csv")
    display(df_aux)

,dataset,r2_test,mse_test,symbolic_model
15,195_auto_price,0.847380,6.095118e+06,"If(highway-mpg>-2.05,0.11*Sub(Mul(0.66*Add(2.35*Atan(4237.12*horsepower),Square(-0.94*peak-rpm)),Ceil(0.86*Ceil(1.44*wheel-base))),Floor(1.00*Add(-6.05*curb-weight,1.94))),1.56*width)"
61,195_auto_price,0.473199,2.117579e+07,"If(curb-weight>2.18,3.58,0.18*Sub(Sub(Sub(Atan(9.81*horsepower),If(highway-mpg>-1.72,-1.53*curb-weight,11.34)),Sinh(-1.22*width)),4.95*Sin(0.44*Log1p(Cos(-1.49*normalized-losses)))))"
77,nikuradse_1,0.985547,3.625310e-04,"Add(1.16*Sinh(0.47*Sub(If(r_k>-0.74,Logabs(0.26),-3.48),1.05*Log1p(4.17*Atan(0.31*r_k)))),Div(Abs(Add(1.12*Sinh(-0.02*log_Re),0.09*Atan(0.74*r_k))),Sqrtabs(0.05*Logistic(Sub(1.00,-2.03*log_Re)))))"
88,nikuradse_1,0.994584,1.280734e-04,"0.77*Sub(Sub(0.69*Sinh(0.47*Add(0.93*Exp(-2.58*r_k),-2.48)),Sqrtabs(0.23)),Mul(Tanh(Sub(If(r_k>-0.71,0.67*r_k,0.32*log_Re),Sub(0.05,-0.25*log_Re))),Log1p(Tanh(Sub(1.65*log_Re,-1.11)))))"
93,nikuradse_1,0.983905,3.719699e-04,"1.71*Sub(1.40*Sub(1.25*Exp(Add(0.35*Sinh(-1.87*r_k),0.25*Log1p(0.47*log_Re))),Exp(0.45)),If(r_k>-0.72,1.46*Add(0.91*Log1p(Log1p(0.26*log_Re)),Sub(Log1p(-0.10*log_Re),Sin(0.56*r_k))),Atan(Sub(Sinh(0.52*log_Re),-0.47))))"
...,...,...,...,...
8324,663_rabe_266,0.999119,1.939278e+00,"1.63*Add(-8.52*Tanh(0.86*Sub(0.20*Log1p(0.95*Log1p(0.26*col_2)),1.05*Sub(1.15*Exp(0.20*col_1),If(col_1>-0.87,0.40,1.40)))),Add(Add(-5.13,If(col_1>-0.87,Atan(-0.56*col_1),7.56*Cosh(-0.25*col_1))),5.16*Div(Sin(0.03*col_2),Abs(-1.56))))"
8349,663_rabe_266,0.998195,2.829246e+00,"3.13*Atan(0.57*Sub(If(col_1>0.23,Exp(0.32*Asin(0.60*col_1)),If(col_2>-1.40,Add(1.82,-0.74),1.03)),0.32*Mul(0.44*Sub(1.04*Log1p(0.50*col_2),Sub(-3.33,-0.15*col_2)),Exp(1.75*Logistic(-1.09*col_1)))))"
8413,4544_GeographicalOriginalofMusic,0.790648,1.643497e-01,"6.56*Sub(Add(Atan(0.05*Sinh(2.10*Sin(0.49*V101))),0.02*V98),Add(Add(If(V39>5.35,V91,-0.11*V42),-0.01*V60),Sub(0.69*Add(Log1p(0.08*V43),Asin(0.05*V45)),Add(Sub(0.04*V103,-0.02*V99),Add(0.02*V71,-0.02*V40)))))"
8451,1030_ERA,0.289275,3.007789e+00,"2.28*Sub(0.15*Sub(0.55*Exp(1.01*Sub(If(in2>-0.23,in2,0.48),Sub(-0.30*in1,0.27*in3))),Logistic(-1.57*in3)),Sub(1.24*Add(0.20*Sub(Add(-2.65,0.45*in4),1.29*Logistic(9.09*in2)),Acos(0.09*in1)),Exp(0.16*in4)))"


NameError: name 'Add' is not defined